In [14]:
%%time
import pal
import json
import random
from tqdm.notebook import tqdm
from pal.prompt import math_prompts, colored_object_prompt, penguin_prompt, date_understanding_prompt, algorithmic_prompt

CPU times: user 1.14 ms, sys: 135 μs, total: 1.27 ms
Wall time: 722 μs


In [22]:
mixed_questions = [
    {"input": """Here is a table where the first line is a header and each subsequent line is a penguin:
name, age, height (cm), weight (kg) 
Freeman, 43, 25, 69
Owen, 28, 61, 42
Jack, 11, 23, 83
For example: The height of Owen is 61 cm. What is the product of the heights of the two youngest penguins?
""".strip(), "target": 1403},
    {"input": """Here is a table where the first line is a header and each subsequent line is an octopus:
name, birthday, height (cm), weight (kg)
Pops, 5/10/2006, 18, 6
Icle, 6/12/2003, 21, 8
Stik, 9/23/2015, 41, 12
For example: The weight of Stik is 12 kg. Today is 8/20/2020. What is the product of the heights of all penguins born over 8 years ago?
""".strip(), "target": 378},
    {"input": """Here is a table where the first line is a header and each subsequent line is a bug:
name, height (cm), weight (kg)
Bobby, 18, 13
Jacky, 17, 17
Smith, 20, 4
For example: The height of Bobby is 18 cm. What is the sum of the weights of the two tallest bugs?
""".strip(), "target": 17},
    {"input": """Today is May 3rd, 1972. I was born exactly 7 years ago. What is the sum of the year I was born and the day I was born?
    """.strip(), "target": 1968},
    {"input": """Here is a table where the first line is a header and each subsequent line is an octopus:
name, birthday, height (cm), weight (kg)
Pops, 5/10/2006, 18, 6
Icle, 6/12/2003, 21, 8
Stik, 9/23/2015, 41, 12
For example: The weight of Stik is 12 kg. Today is 6/12/2016. What is the sum of the weight and height of the octopus who was born exactly 13 years ago?
    """.strip(), "target": 29},
    {"input": """Here is a table where the first line is a header and each subsequent line is a bug:
name, height (cm), weight (kg)
Bobby, 18, 13
Jacky, 17, 17
Smith, 20, 4
For example: The height of Bobby is 18 cm. What is the product of the weights of all three bugs?
    """.strip(), "target": 884},
    {"input": """Here is a table where the first line is a header and each subsequent line is a penguin:
name, age, height (cm), weight (kg) 
Freeman, 43, 25, 69
Owen, 28, 61, 42
Jack, 11, 23, 83
For example: The height of Owen is 61 cm. Find the (positive) difference between the age of the tallest penguin and the age of the shortest penguin.
    """.strip(), "target": 17},
    {"input": """Here is a table where the first line is a header and each subsequent line is a penguin:
name, age, height (cm), weight (kg) 
Fren, 43, 25, 69
Owen, 28, 61, 42
Jack, 11, 23, 83
Cobb, 16, 12, 53
Lobe, 93, 10, 99
For example: The height of Owen is 61 cm. What is the sum of the heights of all penguins who are older than 20?
    """.strip(), "target": 96},
    {"input": """Here is a table where the first line is a header and each subsequent line is a penguin:
name, age, height (cm), weight (kg) 
Fren, 43, 25, 69
Owen, 28, 61, 42
Jack, 11, 23, 83
Cobb, 16, 12, 53
Lobe, 93, 10, 99
For example: The height of Owen is 61 cm. What is the product of the weights of all penguins who are younger than 20?
    """.strip(), "target": 4399},
    {"input": """5 months ago, the date was 9/23/2008. What is the sum of the year and the day today?
    """.strip(), "target": 2032},
    {"input": """Here is a table where the first line is a header and each subsequent line is an octopus:
name, height (cm), weight (kg)
Pops, 18, 6
Icle, 21, 8
Stik, 41, 12
Blob, 23, 9
Kobe, 99, 18
For example: The weight of Stik is 12 kg. What is the result when the weight of the tallest octopus is divided by the weight of the shortest octopus?
    """.strip(), "target": 3},
    {"input": """Here is a table where the first line is a header and each subsequent line is an octopus:
name, height (cm), weight (kg)
Pops, 18, 18
Icle, 21, 8
Stik, 41, 18
Blob, 23, 9
Kobe, 99, 18
For example: The weight of Stik is 18 kg. What is the sum of the heights of all octopus that weigh 18 kg?
    """.strip(), "target": 158},
    {"input": """Here is a table where the first line is a header and each subsequent line is a bug:
name, height (cm), weight (kg), x-coordinate
Bobby, 18, 13, 13
Jacky, 17, 17, 82
Smith, 20, 42, 18
Jonny, 33, 23, 35
For example: The weight of Bobby is 13 kg. What is the sum of the heights of the leftmost and rightmost bugs based on x-coordinate?
    """.strip(), "target": 35},
    {"input": """Here is a table where the first line is a header and each subsequent line is a bug:
name, height (cm), weight (kg), x-coordinate
Bobby, 18, 13, 13
Jacky, 17, 17, 82
Smith, 20, 42, 18
Jonny, 33, 23, 35
For example: The weight of Bobby is 13 kg. What is the sum of the height and weight of the bug with x-coordinate equal to 18?
    """.strip(), "target": 62},
    {"input": """Here is a table where the first line is a header and each subsequent line is a person:
name, height (cm), weight (kg), job
Abe, 193, 58, Doctor
Row, 116, 16, Lawyer
Bay, 152, 46, Doctor
Cho, 123, 25, Gardener
For example: The height of Row is 116 cm. What is the average weight of the doctors?
    """.strip(), "target": 52},
    {"input": """Here is a table where the first line is a header and each subsequent line is a person:
name, height (cm), weight (kg), job
Abe, 193, 58, Doctor
Row, 116, 16, Lawyer
Bay, 152, 46, Doctor
Cho, 123, 25, Gardener
For example: The height of Row is 116 cm. How many kilograms is Cho heavier than Row?
    """.strip(), "target": 9},
    {"input": """Here is a table where the first line is a header and each subsequent line is a person:
name, height (cm), weight (kg), job
Abe, 193, 58, Doctor
Row, 116, 16, Lawyer
Bay, 152, 46, Doctor
Cho, 123, 25, Gardener
For example: The height of Row is 116 cm. What is the sum of the height of the lawyer and the weight of the gardener?
    """.strip(), "target": 141},
    {"input": """Here is a table where the first line is a header and each subsequent line is a person:
name, height (cm), weight (kg), job
Abe, 193, 58, Doctor
Row, 116, 16, Lawyer
Bay, 152, 46, Doctor
Cho, 123, 25, Gardener
For example: The height of Row is 116 cm. What is the product of the weights of the doctors?
    """.strip(), "target": 2668},
    {"input": """Here is a table where the first line is a header and each subsequent line is an octopus:
name, height (cm), weight (kg)
Pops, 18, 18
Icle, 21, 8
Stik, 41, 18
Blob, 23, 9
Kobe, 99, 9
For example: The weight of Stik is 18 kg. What is the product of the heights of all penguins whose weight is 9 kg?
    """.strip(), "target": 2277},
    {"input": """Here is a table where the first line is a header and each subsequent line is an octopus:
name, height (cm), weight (kg)
Pops, 18, 16
Icle, 21, 8
Stik, 41, 12
Blob, 23, 9
Kobe, 99, 20
For example: The weight of Stik is 12 kg. Find the sum of the heights of the lightest and heaviest penguins.
    """.strip(), "target": 120}
                  ]

In [23]:
def getPromptCombos(n, prompts):
    if n > len(prompts):
        return
    if n == 1:
        for i in prompts:
            yield i
        return
    for i in range(len(prompts)):
        for j in getPromptCombos(n - 1, prompts[i + 1:]):
            yield prompts[i] + '\n\n' + j


interface = pal.interface.ProgramInterface(
  model='gpt-4o-mini',
  stop='\n\n\n', # stop generation str for Codex API
  get_answer_expr='solution()' # python expression evaluated after generated code to obtain answer 
)
json_list = []

#with open('../prompt-lib/data/tasks/objectcounting.jsonl', 'r') as json_file:
#    json_list = list(json_file)
for entry in mixed_questions:
    json_list.append(entry)
max_count = 50
#jsonSamples = random.sample(range(0, len(json_list)), max_count)
#jsonSamples = [120, 294, 194, 131, 364, 196, 87]
#jsonSamples = [43]
#print(jsonSamples)
'''
curPrompts = ["1", "2", "3", "4", "5"]
for prompt in getPromptCombos(2, curPrompts):
    print(prompt)
    print("######################")
'''
def voteByMajority(n, question, prompts):
    answers = dict()
    maxNum = -1
    for promptCombo in getPromptCombos(n, prompts):
        answer = interface.run(promptCombo + "\n\n" + "Q: " + question + "\n")
        if answer == None:
            continue
        if answer not in answers.keys():
            answers[answer] = 1
            maxNum = max(maxNum, 1)
        else:
            answers[answer] += 1
            maxNum = max(maxNum, answers[answer])
    return answers

responses = []
for json_pos in tqdm(range(len(json_list)), desc="Going through math questions"):
    result = json_list[json_pos]
    question = result["input"]
    target = result["target"]
    print(result)
    prompt1 = math_prompts.MATH_PROMPT.format(question=question)
    prompt2 = penguin_prompt.PENGUIN_PROMPT.format(question=question)
    prompt3 = date_understanding_prompt.DATE_UNDERSTANDING_PROMPT.format(question=question)
    prompt = prompt1 + '\n' + prompt2 + '\n' + prompt3
    '''
    print("Question: " + str(question))
    print("Desired Result: " + str(result["target"]))
    '''
    answer = None
    try:
        answer = interface.run(prompt)
    except:
        answer = None
    curResponse = {"question": question, "target": target, "received": answer}
    responses.append(curResponse)
numCorrect = 0
curCount = 0
for response in responses:
    #print("Received: " + str(response["received"]))
    #print("Target:", end=' ')
    #print(response["target"])
    #print("")
    '''
    try:
        if abs(response["target"] - response["received"]) < 1e-6:
            numCorrect += 1
        else:
            pass
            print(curCount)
            print("")
    except:
        print(curCount)
        print("Target:", end=' ')
        print(response["target"])
        print("Received:", end=' ')
        print(response["received"])
        print("")
    '''
    if response["target"] == response["received"]:
        numCorrect += 1
    #if response["received"] not in response["target"].keys():
    #    continue
    #if response["target"][response["received"]] == 1:
    #    numCorrect += 1
    curCount += 1
print("Number correct = " + str(numCorrect))

Going through math questions:   0%|          | 0/20 [00:00<?, ?it/s]

{'input': 'Here is a table where the first line is a header and each subsequent line is a penguin:\nname, age, height (cm), weight (kg) \nFreeman, 43, 25, 69\nOwen, 28, 61, 42\nJack, 11, 23, 83\nFor example: The height of Owen is 61 cm. What is the product of the heights of the two youngest penguins?', 'target': 1403}
name 'solution' is not defined
No results was produced. A common reason is that the generated code snippet is not valid or did not return any results.
{'input': 'Here is a table where the first line is a header and each subsequent line is an octopus:\nname, birthday, height (cm), weight (kg)\nPops, 5/10/2006, 18, 6\nIcle, 6/12/2003, 21, 8\nStik, 9/23/2015, 41, 12\nFor example: The weight of Stik is 12 kg. Today is 8/20/2020. What is the product of the heights of all penguins born over 8 years ago?', 'target': 378}
name 'datetime' is not defined
No results was produced. A common reason is that the generated code snippet is not valid or did not return any results.
{'input':

In [24]:
numCorrect = 0
curCount = 0
for response in responses:
    print("Problem " + str(curCount))
    print("Received: " + str(response["received"]))
    print("Target:", end=' ')
    print(response["target"])
    print("")
    curCount += 1
    '''
    try:
        if abs(response["target"] - response["received"]) < 1e-6:
            numCorrect += 1
        else:
            pass
            print(curCount)
            print("")
    except:
        print(curCount)
        print("Target:", end=' ')
        print(response["target"])
        print("Received:", end=' ')
        print(response["received"])
        print("")
    '''
print(numCorrect)

Problem 0
Received: None
Target: 1403

Problem 1
Received: None
Target: 378

Problem 2
Received: 17
Target: 17

Problem 3
Received: 1968
Target: 1968

Problem 4
Received: 29
Target: 29

Problem 5
Received: 884
Target: 884

Problem 6
Received: 17
Target: 17

Problem 7
Received: 17
Target: 96

Problem 8
Received: 17
Target: 4399

Problem 9
Received: 2032
Target: 2032

Problem 10
Received: 3.0
Target: 3

Problem 11
Received: 158
Target: 158

Problem 12
Received: 35
Target: 35

Problem 13
Received: 62
Target: 62

Problem 14
Received: 52.0
Target: 52

Problem 15
Received: 9
Target: 9

Problem 16
Received: 141
Target: 141

Problem 17
Received: 2668
Target: 2668

Problem 18
Received: 2277
Target: 2277

Problem 19
Received: 120
Target: 120

0


In [16]:
print(responses[35])

{'question': 'Mike plays ping pong for 40 minutes.  In the first 20 minutes, he scores 4 points.  In the second 20 minutes, he scores 25% more points.  How many total points did he score?', 'target': 9, 'received': 9.0}


In [17]:
with open('gsm-results-01-pal.json', 'w') as f:
    json.dump(responses, f)

TypeError: Object of type Float is not JSON serializable